In [9]:
import pygame
import math
import colorsys
import cv2
import numpy as np
from IPython.display import clear_output

# Initialize Pygame
pygame.init()

# Setup
width, height = 800, 800
screen = pygame.Surface((width, height))  # Using Surface instead of display.set_mode()

# Video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('ColorfulSpiralPattern.mp4', fourcc, 60.0, (width, height))

# Constants
PHI = (1 + math.sqrt(5)) / 2
GOLDEN_ANGLE = math.pi * 2 * (1 - 1 / PHI)

# Animation loop
t = 0
while t < 10:  # 10 seconds of animation 
    screen.fill((0, 0, 0))

    max_points = 500
    base_radius = min(width, height) / 4

    for i in range(max_points):
        distance = (i / max_points) * base_radius
        angle = i * GOLDEN_ANGLE + t

        wave = math.sin(i * 0.1 + t) * 20
        x = math.cos(angle) * (distance + wave) + width/2
        y = math.sin(angle) * (distance + wave) + height/2

        size = 2 + math.sin(i * 0.1 + t) * 2
        hue = (i * 2 + t * 50) % 360 / 360
        light = 0.5 + math.sin(i * 0.1 + t) * 0.2
        rgb = [int(x * 255) for x in colorsys.hls_to_rgb(hue, light, 1)]

        pygame.draw.circle(screen, rgb, (int(x), int(y)), int(size))

        if i > 0 and i % 5 == 0:
            prev_dist = ((i - 5) / max_points) * base_radius
            prev_angle = (i - 5) * GOLDEN_ANGLE + t
            prev_x = math.cos(prev_angle) * prev_dist + width/2
            prev_y = math.sin(prev_angle) * prev_dist + height/2
            pygame.draw.line(screen, (*rgb, 77), (x, y), (prev_x, prev_y), 1)

    # Save frame to video
    pixels = pygame.surfarray.array3d(screen)
    pixels = pixels.swapaxes(0,1)
    pixels = cv2.cvtColor(pixels, cv2.COLOR_RGB2BGR)
    video.write(pixels)

    t += 0.02
    clear_output(wait=True)
    print(f"Rendering: {int(t*10)}%")

video.release()
pygame.quit()
print("Video saved as 'ColorfulSpiralPattern.mp4'")




Rendering: 100%
Video saved as 'ColorfulSpiralPattern.mp4'
